In [ ]:
import pandas as pd
import json
from pandas import json_normalize

In [ ]:
with open("csm_data/Registration_data_csm.json", "r") as f:
    regdata = json.load(f)

In [ ]:
columns = list(regdata[0].keys())
columns

In [ ]:
# Normalize the nested JSON
df = json_normalize(regdata)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df["ShowRef"].unique()

# Find Visitor in 24 and 25

- USe the email address
- create dataframe of 25 with Visitors which repeat
- Remove Badge Type not valid like 

In [ ]:
shows_24 = ["BVA2024", "DL24", "DCWL24", "CEEL24", "CCSEL24", "BDAWL24"]
shows_25 = ["BDAWL25", "CEEL25", "CCSEL25", "DCWL25", "DL25"]

In [ ]:
df_reg_24 = df[df.ShowRef.isin(shows_24)]
df_reg_25 = df[df.ShowRef.isin(shows_25)]
len(df_reg_24), len(df_reg_25)

In [ ]:
email_24_list = list(df_reg_24["Email"].unique())
email_25_list = list(df_reg_25["Email"].unique())

In [ ]:
email_24_25 = set(email_25_list).intersection(set(email_24_list))

In [ ]:
len(email_24_25)

In [ ]:
dfreg_24_25 = df_reg_25[df_reg_25["Email"].isin(email_24_25)]
len(dfreg_24_25)

In [ ]:
dfreg_24_25 = pd.merge(
    dfreg_24_25, df_reg_24[["Email", "ShowRef", "BadgeId"]], on="Email", how="inner"
)
len(dfreg_24_25)

In [ ]:
dfreg_24_25.isnull().sum()

In [ ]:
dfreg_24_25.to_csv("csv/Registration_data_24_25_both.csv", index=False)
len(dfreg_24_25)

In [ ]:
dfreg_24_25["BadgeType"].unique()

In [ ]:
badgeIDs_list = ["VIP", "Visitor", "Commercial Visitor"]

In [ ]:
dfreg_24_25_good_badge_id = dfreg_24_25[dfreg_24_25["BadgeType"].isin(badgeIDs_list)]

In [ ]:
len(dfreg_24_25_good_badge_id)

In [ ]:
dfreg_24_25_good_badge_id.to_csv(
    "csv/Registration_data_24_25_both_only_valid_badge_type.csv", index=False
)

In [ ]:
dfreg_24_25_good_badge_id.head()

# Create a list of BadgeID of those Visitor which are registered to both events

In [ ]:
list_badges_24_25 = list(dfreg_24_25_good_badge_id["BadgeId_y"].unique())
len(list_badges_24_25)

In [ ]:
list_badges_24_25[0]

# Badge Data

In [ ]:
path_seminar_reference_24 = "csv/tech london 2024 seminars reference.csv"
path_badge_scan_24 = "csv/tech london 2024 seminars.csv"

In [ ]:
reference_data_24 = pd.read_csv(path_seminar_reference_24)
scans_data_24 = pd.read_csv(path_badge_scan_24)

In [ ]:
reference_data_24.head(2)

In [ ]:
scans_data_24.head(2)

In [ ]:
commom_sessions = set(reference_data_24["Short Name"].unique()).intersection(
    set(scans_data_24["Short Name"].unique())
)

In [ ]:
scans_data_24.columns

In [ ]:
badge_scan_list = set(list(scans_data_24["Badge Id"].unique())).intersection(
    set(list_badges_24_25)
)
len(badge_scan_list)

### Extract activity in 2024 of Visitors which are registered in both years

In [ ]:
scans_data_24_25 = scans_data_24[scans_data_24["Badge Id"].isin(badge_scan_list)]
len(scans_data_24_25)

In [ ]:
reference_data_24.columns

In [ ]:
scans_data_24_25.columns

In [ ]:
scans_data_24_25_enriched = pd.merge(
    scans_data_24_25[["Short Name", "Badge Id"]],
    reference_data_24[["Short Name", "Seminar Name"]],
    on=["Short Name"],
    how="inner",
)
len(scans_data_24_25_enriched)

In [ ]:
scans_data_24_25_enriched.to_csv("csv/scans_data_24_25_enriched.csv", index=False)

# include scan data in Demograpifc Data 2025



In [ ]:
scans_data_24_25_enriched.columns

In [ ]:
len(scans_data_24_25_enriched["Badge Id"].unique()), len(
    scans_data_24_25_enriched["Badge Id"]
)

In [ ]:
# Group by 'Badge Id' and concatenate 'Seminar Name'
grouped_df = (
    scans_data_24_25_enriched.groupby("Badge Id")["Seminar Name"]
    .apply(", ".join)
    .reset_index()
)

# Rename the columns
grouped_df.columns = ["Badge Id", "Concatenated Seminar Names"]

# Output the new DataFrame
grouped_df.head(5)

In [ ]:
grouped_df.to_csv("csv/scans_data_24_25_enriched_grouped.csv", index=False)

In [ ]:
# Merging the DataFrames on 'BadgeId_y' and 'Badge Id'
merged_df = pd.merge(
    dfreg_24_25_good_badge_id,
    grouped_df,
    left_on="BadgeId_y",
    right_on="Badge Id",
    how="left",
)

# Dropping the redundant 'Badge Id' column if needed
merged_df = merged_df.drop(columns=["Badge Id"])

# Output the merged DataFrame(merged_df)

merged_df.head()

In [ ]:
merged_df.columns

In [ ]:
merged_df = merged_df[
    [
        "Id",
        "Title",
        "Forename",
        "Surname",
        "Email",
        "Tel",
        "Mobile",
        "Fax",
        "Company",
        "JobTitle",
        "Addr1",
        "Addr2",
        "Addr3",
        "Town",
        "County",
        "Postcode",
        "Country",
        "Status",
        "BadgeType",
        "ShowRef_x",
        "RegistrationDate",
        "BadgeId_x",
        "RegCode",
        "Source",
        "Attended",
        "SegmentFromReg",
        "Upgrade",
        "Concatenated Seminar Names",
    ]
]

In [ ]:
merged_df.columns = [
    "Id",
    "Title",
    "Forename",
    "Surname",
    "Email",
    "Tel",
    "Mobile",
    "Fax",
    "Company",
    "JobTitle",
    "Addr1",
    "Addr2",
    "Addr3",
    "Town",
    "County",
    "Postcode",
    "Country",
    "Status",
    "BadgeType",
    "ShowRef",
    "RegistrationDate",
    "BadgeId",
    "RegCode",
    "Source",
    "Attended",
    "SegmentFromReg",
    "Upgrade",
    "Concatenated Seminar Names",
]

In [ ]:
merged_df = merged_df[["Email", "ShowRef", "BadgeId", "Concatenated Seminar Names"]]

In [ ]:
merged_df.to_csv(
    "csv/Registration_data_24_25_both_only_valid_badge_type_with_seminars.csv",
    index=False,
)